In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
import torch.optim as optim


import torch
import matplotlib.pyplot as plt 
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from torchvision.datasets import MNIST
from PIL import Image
import random
import IPython


num_epochs = 3
batch_size = 100
learning_rate = 0.1

# Image Preprocessing 


In [ ]:
class ColorsA(MNIST):
    def __init__(self, root, train=True, download=False, transform=None, target_transform=None):
        super(ColorsA, self).__init__(root, train=train, download=download, transform=transform, target_transform=target_transform)
        self.label_to_color_dict = {}
        colors = [(160, 0, 160), (200, 255, 0), (0, 0, 100), (0, 255, 0), (0, 0, 255), (120, 120, 120),(0, 128, 128),(255, 165, 165), (100, 40, 40), (130, 157, 220)]
        for i in range(10):
          self.label_to_color_dict[i] = colors[i]

    def __getitem__(self, index):
        img, label = self.data[index], self.targets[index]
        img = img.numpy()
        color = self.label_to_color_dict[label.item()]
        img = np.stack((img, img, img), axis=2)
        img = np.where(img > 0, color, img)
        img = img.astype(np.uint8)
        img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            label = self.target_transform(label)

        return img, label

class ColorsB(MNIST):
    def __init__(self, root, train=True, download=False, transform=None, target_transform=None):
        super(ColorsB, self).__init__(root, train=train, download=download, transform=transform, target_transform=target_transform)

        self.label_to_color_dict = {}
        colors = [(130, 157, 220),(100, 40, 40),(255, 165, 165),(0, 128, 128),(120, 120, 120), (0, 0, 255),(0, 255, 0), (0, 0, 100),(200, 255, 0),(160, 0, 160)]
        for i in range(10):
          self.label_to_color_dict[i] = colors[i]

    def __getitem__(self, index):
        img, label = self.data[index], self.targets[index]
        img = img.numpy()
        color = self.label_to_color_dict[label.item()]
        img = np.stack((img, img, img), axis=2)
        img = np.where(img > 0, color, img)
        img = img.astype(np.uint8)
        img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            label = self.target_transform(label)

        return img, label

In [ ]:
def to_cuda(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return x

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [ ]:

transform = transforms.Compose([transforms.ToTensor()])
colored_trainA = ColorsA(root='./data', train=True, download=True, transform=transform)
train_loader_A = torch.utils.data.DataLoader(dataset=colored_trainA, batch_size=120, shuffle=True, num_workers = 4,pin_memory=True)

colored_trainB = ColorsB(root='./data', train=True, download=True, transform=transform)
train_loader_B = torch.utils.data.DataLoader(dataset=colored_trainB, batch_size=120, shuffle=True, num_workers = 4, pin_memory=True)


In [ ]:
import math
import torchvision.utils as vutils
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def generate_imgs(x, y, xy_gen, yx_gen, epoch=0):
   
    xy_gen.eval()
    yx_gen.eval()

    y_fake = xy_gen(x)
    x_fake = yx_gen(y)

    x_imgs = torch.zeros((x.shape[0] * 2, 3, x.shape[2], x.shape[3]))
    y_imgs = torch.zeros((y.shape[0] * 2, 3, y.shape[2], y.shape[3]))

    even_idx = torch.arange(start=0, end=x.shape[0] * 2, step=2)
    odd_idx = torch.arange(start=1, end=x.shape[0] * 2, step=2)

    x_imgs[even_idx] = x.cpu()
    x_imgs[odd_idx] = y_fake.cpu()

    y_imgs[even_idx] = y.cpu()
    y_imgs[odd_idx] = x_fake.cpu()

    rows = math.ceil((x.shape[0] * 2) ** 0.5)

    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(16,9))

    x_imgs_ = torchvision.utils.make_grid(x_imgs, nrow=rows).permute(1,2,0).numpy()*255
    x_imgs_ = x_imgs_.astype(np.uint8)
    ax1.imshow(Image.fromarray(x_imgs_))
    ax1.set_xticks([])
    ax1.set_yticks([])


    y_imgs_ = torchvision.utils.make_grid(y_imgs, nrow=rows).permute(1,2,0).numpy()*255
    y_imgs_ = y_imgs_.astype(np.uint8)
    ax2.imshow(Image.fromarray(y_imgs_))
    ax2.set_xticks([])
    ax2.set_yticks([])

    plt.show()



In [ ]:

class Discriminator_A(nn.Module):
    def __init__(self):
        super(Discriminator_A, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.fc1 = nn.Linear(128 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()
        self.leakyrelu = nn.LeakyReLU(negative_slope=0.2)

        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 128 * 7 * 7)
        x = self.leakyrelu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


class Discriminator_B(nn.Module):
    def __init__(self):
      super(Discriminator_B, self).__init__()
      
      self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
      self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
      self.fc1 = nn.Linear(128 * 7 * 7, 256)
      self.fc2 = nn.Linear(256, 1)     
      self.sigmoid = nn.Sigmoid()
      self.leakyrelu = nn.LeakyReLU(negative_slope=0.2)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 128 * 7 * 7)
        x = self.leakyrelu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x



class Generator_A(nn.Module):
    def __init__(self):
        super(Generator_A, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 3, kernel_size=4, stride=1, padding=1)

        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x


class  Generator_B(nn.Module):
    def __init__(self):
        super( Generator_B, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 3, kernel_size=4, stride=1, padding=1)

        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [ ]:


ab_gen = Generator_A().to(device)
ba_gen = Generator_B().to(device)
a_disc = Discriminator_A().to(device)
b_disc =  Discriminator_B().to(device)

# Define Optimizers

dis_a_opt = optim.Adam(a_disc.parameters(), lr=0.001)
dis_b_opt = optim.Adam(b_disc.parameters(), lr=0.001)
gen_ab_opt = optim.Adam(ab_gen.parameters(), lr=0.001)
gen_ba_opt = optim.Adam(ba_gen.parameters(), lr=0.001)

criterion = nn.BCELoss()


a_fixed, _ = next(iter(train_loader_A))
b_fixed, _ = next(iter(train_loader_B))
a_fixed = a_fixed.to(device)
b_fixed = b_fixed.to(device)


def train(epochs):
  for epoch in range(epochs):
      print(f"Epoch: {epoch+1} / {epochs}")
      print("-"*90)

      running_ab_gen_loss = 0.0
      running_ba_gen_loss = 0.0
      running_a_disc_loss = 0.0
      running_b_disc_loss = 0.0

      for i, (a_data, b_data) in enumerate(zip(train_loader_A, train_loader_B)):

          # Loading data
          a_real, _ = a_data
          b_real, _ = b_data

          a_real = a_real.to(device)
          b_real = b_real.to(device)

          batch_size = a_real.size(0)

          real_labels = torch.ones(batch_size).to(device)
          fake_labels = torch.zeros(batch_size).to(device)

          # ======================================= Training A Discriminator =======================================
          a_disc.train()
          ba_gen.train(False)

          ## real images
          a_real_out = a_disc(a_real)
          a_disc_real_loss = criterion(a_real_out.squeeze(1), real_labels)

          ## fake images
          a_fake = ba_gen(b_real)
          a_fake_out = a_disc(a_fake)
          a_disc_fake_loss = criterion(a_fake_out.squeeze(1), fake_labels)
          
          a_disc_total_loss = a_disc_real_loss + a_disc_fake_loss
          a_disc.zero_grad()
          running_a_disc_loss += a_disc_total_loss.item()           
          a_disc_total_loss.backward()
          dis_a_opt.step()

          # ======================================= Training B -> A Generator =======================================
          a_disc.train(False)
          ba_gen.train()

          a_fake = ba_gen(b_real)
          a_fake_out = a_disc(a_fake)
          ba_gen_loss = criterion(a_fake_out.squeeze(1), real_labels)
          running_ba_gen_loss += ba_gen_loss.item()

          a_disc.zero_grad()
          ba_gen.zero_grad()
          ba_gen_loss.backward()
          gen_ba_opt.step()

          ba_gen.train(False)
          # ======================================= Training B Discriminator =======================================
          b_disc.train()
          ab_gen.train(False)

          ## real images
          b_real_out = b_disc(b_real)
          b_disc_real_loss = criterion(b_real_out.squeeze(1), real_labels)

          ## fake images
          b_fake = ab_gen(a_real)
          b_fake_out = b_disc(b_fake)
          b_disc_fake_loss = criterion(b_fake_out.squeeze(1), fake_labels)

          b_disc_total_loss = b_disc_real_loss + b_disc_fake_loss
          b_disc.zero_grad()
          running_b_disc_loss += b_disc_total_loss.item()           
          b_disc_total_loss.backward()
          dis_b_opt.step()

          # ======================================= Training A -> B Generator =======================================
          b_disc.train(False)
          ab_gen.train()

          b_fake = ab_gen(a_real)
          b_fake_out = b_disc(b_fake)
          ab_gen_loss = criterion(b_fake_out.squeeze(1), real_labels)
          running_ab_gen_loss += ab_gen_loss.item()

          b_disc.zero_grad()
          ab_gen.zero_grad()
          ab_gen_loss.backward()
          gen_ab_opt.step()

          ab_gen.train(False)
              
      print(f"avg discriminator A loss: {round(running_a_disc_loss/len(train_loader_A),4)}")
      print(f"avg discriminator B loss: {round(running_b_disc_loss/len(train_loader_A),4)}")
      print(f"avg generator from A to B loss: {round(running_ab_gen_loss/len(train_loader_A),4)}")
      print(f"avg generator from B to A loss: {round(running_ba_gen_loss/len(train_loader_A),4)}")
      
      print("-"*90)
      generate_imgs(a_fixed, b_fixed, ab_gen, ba_gen)

  generate_imgs(a_fixed, b_fixed, ab_gen, ba_gen)


train(9)

In [ ]:
torch.save(ab_gen.state_dict(), 'ab_gen.pkl')
torch.save(ba_gen.state_dict(), 'ba_gen.pkl')
torch.save(a_disc.state_dict(), 'a_disc.pkl')
torch.save(b_disc.state_dict(), 'b_disc.pkl')